
# Paranoid analysis of iMessage chats with OpenAI, LlamaIndex & DuckDB

Can I safely combine my local personal data with a public large language model to understand my texting behaviour? 

![](docs/arch.png)

A project combining natural language and generative AI models to explore my private data without sharing (too much of) my personal life with the robots.

# iMessage analysis - Prepare data

## Notebook for Fitbit data analysis with DuckDB
Expected to be run in a notebook, DuckDB’s Python client can be used [directly in Jupyter notebook](https://duckdb.org/docs/guides/python/jupyter)

First step is import the relevant librariesSet and configure to directly output data to Pandas and to simplify the output that is printed to the notebook.

In [ ]:
import duckdb
import pandas as pd

%load_ext sql
%sql duckdb:///myduck.db

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

## Loading data

If you are on a Mac, logged into your iCloud account you can copy the local iMessage SQLite database and load it.

⏩ You can skip these steps - jump to [Generative AI with Pandas and LlamaIndex](#genai) if you just want to analyse some sample data

### SQLite Scanner DuckDB extension

Load [SQLite Scanner DuckDB extension](https://duckdb.org/docs/extensions/sqlite_scanner) is a loadable extension that allows DuckDB to directly read data from a SQLite database such as iMessage.

In [ ]:
%%sql
INSTALL sqlite_scanner;
LOAD sqlite_scanner;

### Copy your iMessage
We now copy the iMessage SQLite databe. You'll need to be on on a Mac, logged into your Imessage account which has a copy of your messages. 

If you recieve the error `Operation not permitted` you may need to run the command in a terminal and accept the prompts to interact with privilaged files.

In [ ]:
! cp ~/Library/Messages/chat.db ./sql/chat.db

### Attach DuckDB to SQLite database

We will open the iMessage SQLite database, with the [attach](https://duckdb.org/docs/sql/statements/attach.html) command. This will open the SQLite database file `./sql/chat.db` in the schema namespace `chat_sqlite`
chat

In [ ]:
%%sql
DETACH DATABASE IF EXISTS chat_sqlite;

ATTACH './sql/chat.db' as chat_sqlite (TYPE sqlite,  READ_ONLY TRUE);

### Load messages into table
We create the `chat_messages` DuckDB table by joining three tables from the iMessage SQLite database. 

We also 
- determine the message time by evaluating the interval (number of seconds since EPOC of `2001-01-01`)
- extract the phone number country calling code (eg, `+1`, `+61`) 
- redact phone number like `+61412341234` to `+614...41234`

In [ ]:
%%sql
CREATE OR REPLACE TABLE chat_messages
AS
SELECT TIMESTAMP '2001-01-01' + INTERVAL (msg.date / 1000000000) seconds + INTERVAL 10 HOURS as message_date,
msg.text,
msg.attributedBody,
msg.is_from_me,
CASE WHEN ct.chat_identifier like '+1%' then SUBSTRING(ct.chat_identifier, 1, 2) when ct.chat_identifier like '+%' then SUBSTRING(ct.chat_identifier, 1, 3) end as phone_country_calling_code,
CASE WHEN ct.chat_identifier not like '+%' then ct.chat_identifier end as email,
regexp_replace(ct.chat_identifier, '^(\+[0-9][0-9][0-9])([0-9][0-9][0-9])', '\1...\3') as phone_number
FROM chat_sqlite.chat ct
JOIN chat_sqlite.chat_message_join cmj ON ct."ROWID" = cmj.chat_id
JOIN chat_sqlite.message msg ON cmj.message_id = msg."ROWID";

In [ ]:
%%sql
chat_messages_df <<
  SELECT *
  FROM chat_messages
  ORDER BY message_date DESC;

### Decoding attributedBody

The iMessage database has a mixure of encoding formats, with older messages as plain text in the `text` field, with newer messages encoded in the `attributedBody` field. 

Sometime around November 2022 the messages started coming in in new format which migh be related to a message upgrade related to the release of iOS 16.

I'm thankful to the [iMessage-Tools](https://github.com/my-other-github-account/imessage_tools/) project which had the logic to extract the text content is hidden within the `attributedBody` field. The `decode_message` utility function extracts the text regardsless of format.

In [ ]:
import re

def decode_message(row):
  msg_text = row['text']
  msg_attributed_body = row['attributedBody']

  # Logic from https://github.com/my-other-github-account/imessage_tools
  body=''
  if not msg_text:
    body = msg_text
  elif msg_attributed_body is None:
    body = ''
  else:
    try:
      msg_attributed_body = msg_attributed_body.decode('utf-8', errors='replace')
    except AttributeError as err:
      pass

    if "NSNumber" in str(msg_attributed_body):
      msg_attributed_body = str(msg_attributed_body).split("NSNumber")[0]
      if "NSString" in msg_attributed_body:
        msg_attributed_body = str(msg_attributed_body).split("NSString")[1]
        if "NSDictionary" in msg_attributed_body:
          msg_attributed_body = str(msg_attributed_body).split("NSDictionary")[0]
          msg_attributed_body = msg_attributed_body[6:-12]
          body = msg_attributed_body

  body = re.sub(r'\n', ' ', body)
  return body

### Inline message extraction
We'll use a the [pandas apply()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) method to apply the `decode_message` function to the DataFrame. We will also drop some unnessary columns

In [ ]:
chat_messages_df['message_text'] = chat_messages_df.apply(decode_message, axis=1)


In [ ]:
chat_messages_df = chat_messages_df.drop(['text', 'attributedBody'], axis=1, errors='ignore')

In [ ]:
chat_messages_df.head(10)

### Export the DataFrame to disk (optional)
This is optional - we can export the DataFrame to a CSV file.

In [ ]:
%%sql
CREATE OR REPLACE TABLE SAMPLE_CHAT_MESSAGES
AS
SELECT message_date, is_from_me, phone_country_calling_code, phone_number, email
, translate(message_text, 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789', 'xxxxxxxxxxxxxxxxxxxxxxxxxxXXXXXXXXXXXXXXXXXXXXXXXXXXnnnnnnnnnn')  as message_text                
FROM chat_messages_df 
WHERE EMAIL IS NULL;


In [ ]:
%%sql
COPY (
  SELECT *
  FROM SAMPLE_CHAT_MESSAGES
  USING SAMPLE 50
  ) 
TO 'sample_chat_messages.csv' (HEADER, DELIMITER ',');

# Generative AI with Pandas and LlamaIndex

<a id='genai'></a>


We'll be using [LlamaIndex](https://www.llamaindex.ai/) -  a flexible data framework for connecting custom data sources to large language models.

You have now either extracted your iMessages, or you are happy to use some sample data. We'll first load the `chat_messages_df` dataframe containing messages to process.

In [ ]:
%%sql
chat_messages_df <<
  SELECT * 
  FROM read_csv_auto('sample_chat_messages.csv');

### "Man in the Middle" HTTPS proxy - mitmproxy (optional)
This is an optional step if you want to inspect the traffic. Install [mitmproxy](https://mitmproxy.org/) which is a free and open source HTTPS proxy - used to intercept and inspect SSL/TLS-protected traffic.

If you want to view the traffice, start `mitmweb` and set the following environment variables.

⏩ However, you can skip these steps - jump to [LlamaIndex](#llamaindex) to analyse data.

In [ ]:
import os
os.environ['http_proxy'] = "http://127.0.0.1:8080" 
os.environ['https_proxy'] = "https://127.0.0.1:8080" 
os.environ['REQUESTS_CA_BUNDLE'] = "/Users/saubury/.mitmproxy/mitmproxy-ca-cert.pem" 

## LlamaIndex

<a id='llamaindex'></a>


In [ ]:
import pandas as pd
from llama_index.query_engine.pandas_query_engine import PandasQueryEngine
import openai
import configparser

config = configparser.ConfigParser()
config.read('notebook.cfg')
openai_api_token = config.get('openai', 'api_token')
openai.api_key =  openai_api_token

### 1. Most frequent contact

In [ ]:
query_engine = PandasQueryEngine(df=chat_messages_df[['phone_number']], verbose=False)
response = query_engine.query("What is the most frequent phone_number?")
print(response)

### 2. Plot a bar chart showing the frequency by phone number

In [ ]:
query_engine = PandasQueryEngine(df=chat_messages_df[['phone_number']], verbose=False)

response = query_engine.query(
    """Create a plot a bar chart showing the frequency of top eight phone_numbers. 
    Add a title. 
    The X axis lables should be at a 45 degree angle.
    Use a differnt colour for each bar.""",
)

### 3. Distribution of the message activity through the day

In [ ]:
query_engine = PandasQueryEngine(df=chat_messages_df[['message_date']], verbose=False)

response = query_engine.query(
    """Extract hour of day from message_date. 
    Visualize a distribution of the hour extracted from message_date. 
    Add a title and label the axis. 
    Use colors. 
    Add a gap between bars. 
    Colour the bars with an hour of 5 in red and the rest in blue.""",
)

### 4. Distribution of the length of message

In [ ]:
query_engine = PandasQueryEngine(df=chat_messages_df[['message_text']], verbose=False)

response = query_engine.query(
    """Visualize a distribution of the length of message_text. 
    Use a logerithmic scale.  
    Add a title and label both axis. 
    Add a space between bars."""
)

In [ ]:
query_engine = PandasQueryEngine(df=chat_messages_df[['is_from_me']], verbose=False)

response = query_engine.query(
    """Visualize a pie chart of the proption of is_from_me. 
    Label the value 0 as 'inbound'. 
    Add a percentage rounded to 1 decimal places"""
)